In [75]:
import json
import openai
from typing import List, Dict, Any, Optional
import requests
import os
from pydantic import BaseModel, Field
import openai
from dotenv import load_dotenv

load_dotenv()





class RetrievalRequest(BaseModel):
    chat_history: List[Dict[str, str]]
    top_k: int = 5


# Rest of your existing code...
class Settings:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    BACKEND_URL = os.getenv("BACKEND_URL")

settings = Settings()

class ActionCollectiveRequest(BaseModel):
    """Discover and execute actions to solve a task that may require computation or retrieval of information"""
    thought: str = Field(..., description="Thoughts about the task that needs to be solved and how to resolve it.")
    is_action_needed: bool = Field(..., description="Whether an action is needed to resolve the task.")
    tool_description: str = Field(..., description="Description of the tool that can be used to resolve the task.")


class ActionDataGenerator(BaseModel):
    """Schema for defining a single computational action that the LLM can create and execute"""
    input_json_schema: str = Field(
        ..., 
        description="JSON Schema defining the expected input parameters for the action function"
    )
    output_json_schema: str = Field(
        ..., 
        description="JSON Schema defining the expected return value from the action function"
    )
    code: str = Field(
        ..., 
        description="""Python code implementing a function named 'action' that takes parameters matching the input_json_schema.
It must return a value matching the output_json_schema.
You only have access to the following libraries: numpy, requests. You must import them at the top of the file if you would like to use them.
Example delimited by triple backticks:
```
def action(input1: int, input2: int) -> dict:
    # sum of the two inputs

    return {"result": input1 + input2}
```"""
    )
    test: str = Field(
        ..., 
        description="""This is code that will be appended to the previous code. It will be used to test the action function.
It must use assertions to validate the action function works correctly with static test inputs derived from the current task.
The test should NEVER be quantitative, it should only be qualitative. If it does need to be quantitative than it should be code not hard values.
Example delimited by triple backticks:
```
i1 = 7
i2 = 8
sum = action(input1, input2)
assert isinstance(sum["result"], int)
assert sum["result"] == i1 + i2
```"""
    )

class ActionData(ActionDataGenerator):
    chat_history: List[Dict[str, str]]  # List of chat messages

class ActionClient:
    def __init__(self):
        self.client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        self.chat_history: List[Dict[str, str]] = []

    def retrieve_actions(self, k: int = 5) -> list[ActionData]:
        response = requests.post(
            f"{settings.BACKEND_URL}/retrieve_actions",
            json={"chat_history": self.chat_history, "top_k": k},
        )
        # TODO: can query with the action though included in the chat history\
        response = response.json()
        print("\n\nretrieve_actions:\n", json.dumps(response, indent=4))
        retrieval_response = [ActionData.model_validate(action) for action in response]
        return retrieval_response


    def produce_action_thought(self) -> ActionCollectiveRequest:
        completion = self.client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=self.chat_history,
            response_format=ActionCollectiveRequest,
        )

        parsed_response = completion.choices[0].message.parsed

        print("\n\nproduce_action_thought:\n", parsed_response)

        # if no action is needed
        if not parsed_response or not parsed_response.is_action_needed:
            raise Exception("No action thought created")

        return parsed_response
    
    def create_action_data(self) -> ActionData:
        response = self.client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=self.chat_history,
            response_format=ActionDataGenerator,
        )

        action_data_generator = response.choices[0].message.parsed

        print("\n\ncreate_action_data:\n", action_data_generator.model_dump_json(indent=4))

        if not action_data_generator:
            raise Exception("No action created")
        
        action_data_generator.code = action_data_generator.code.replace("```python", "").replace("```", "").strip()
        action_data_generator.test = action_data_generator.test.replace("```python", "").replace("```", "").strip()
        
        action_data = ActionData.model_validate({**action_data_generator.model_dump(), "chat_history": self.chat_history})
        
        return action_data
    
    def upload_action(self, action_schema: ActionData) -> bool:
        response = requests.post(
            f"{settings.BACKEND_URL}/submit_action",
            json=action_schema.model_dump(),
        )
        updated_action = response.json()
        print("\n\nupdated_action:\n", updated_action)
        if not updated_action:
            raise Exception("Failed to upload action")
        return updated_action

    def pipe(self, user_message: str) -> Optional[str]:
        # Add user message to chat history
        self.chat_history.append({"role": "user", "content": user_message})

        # First, try to retrieve existing actions
        retrieval_response = self.retrieve_actions(k=5)

        # if action is retrieved
        if retrieval_response:
            return retrieval_response[0].code

        # If suitable actions found, use them
        # TODO: Implement logic to determine if retrieved actions are suitable

        action_thought = self.produce_action_thought()
        
        self.chat_history.append(
            {
                "role": "assistant",
                "content": f"""
                            THOUGHT:
                                {action_thought.thought}
                            TOOL DESCRIPTION:
                                {action_thought.tool_description}"""
            }
        )

        action_schema = self.create_action_data()
        try:
            complete_test = action_schema.code + "\n" + action_schema.test
            print("\n\nexecuting:\n\n", complete_test)
            exec(complete_test)
            print("\n\nPASSED")
        except Exception as e:
            print("\n\nFAILED", e)
            return None

        self.upload_action(action_schema)

        return action_schema.code

In [76]:
client = ActionClient()
response = client.pipe(
"""Please perform the matrix multiplication of A x B and return the result, here are the variables:
A = [[1, 2, 3, 4, 5],
        [6, 7, 7, 9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 7, 19, 20],
        [21, 22, 23, 24, 25]]
B = [[1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 7, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]]"""
)
print(response)






retrieve_actions:
 [
    {
        "input_json_schema": "{\"A\": [[1, 2, 3, 4, 5], [6, 7, 7, 9, 10], [11, 12, 13, 14, 15], [16, 17, 7, 19, 20], [21, 22, 23, 24, 25]], \"B\": [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 7, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]}",
        "output_json_schema": "{\"result\": [[365, 385, 430, 475, 520], [174, 194, 187, 223, 247], [436, 467, 467, 596, 635], [507, 550, 485, 591, 635], [1205, 1290, 1485, 1510, 1565]]}",
        "code": "import numpy as np\n\ndef action(A: list, B: list) -> dict:\n    # Convert lists to numpy arrays\n    A_array = np.array(A)\n    B_array = np.array(B)\n    # Perform matrix multiplication\n    result_array = np.dot(A_array, B_array)\n    # Convert the result back to a list and return it in a dictionary\n    return {\"result\": result_array.tolist()}",
        "test": "A = [[1, 2, 3, 4, 5], [6, 7, 7, 9, 10], [11, 12, 13, 14, 15], [16, 17, 7, 19, 20], [21, 22, 23, 24, 25]]\nB = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]

In [67]:
class asd(BaseModel):
    a: str
    b: str

class asd2(asd):
    c: str

new_asd2 = asd2.model_validate({"a": "1", "b": "2", "c": "3"})
new_asd = asd.model_validate(new_asd2)
print(new_asd.a)


1
